In [22]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import plotly.express as px

from datetime import datetime
from scipy import stats

#Set Display options
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 50)

In [23]:
#Pick Season
season = "2020-21"

#Import Files
all_players = pd.read_csv (r'./data/' + season + '/Merged_data/all_players_merged.csv', parse_dates= ["news_added"])
#all_teams = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_teams.csv')
all_events = pd.read_csv (r'./data/' + season + '/Cleaned_data/all_events.csv', parse_dates= ["deadline_time"])
player_types = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_types.csv')
game_phases = pd.read_csv(r'./data/' + season + '/Cleaned_data/game_phases.csv')

player_season_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_season_history.csv')
player_gameweek_history = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_gameweek_history.csv')
#player_gameweek_history = pd.read_csv(r'./data/2019-20/player_past_history/player_gameweek_history_raw.csv')
player_future_fixture = pd.read_csv(r'./data/' + season + '/Cleaned_data/player_future_fixture.csv', parse_dates = ["kickoff_time"])



In [24]:
player_future_fixture.head()

,fixture_id,fixture_code,home_team_id,team_h_score,away_team_id,team_a_score,event,finished,minutes,provisional_start_time,kickoff_time,event_name,is_home,difficulty,player_id,opponent_team_id,team_id
0,64,2128351,13,NaN,1,NaN,7.0,False,0,False,2020-11-01 16:30:00,Gameweek 7,False,4,1,13,1
1,69,2128356,1,NaN,2,NaN,8.0,False,0,False,2020-11-07 15:00:00,Gameweek 8,True,3,1,2,1
2,82,2128369,10,NaN,1,NaN,9.0,False,0,False,2020-11-22 16:30:00,Gameweek 9,False,3,1,10,1
3,89,2128376,1,NaN,20,NaN,10.0,False,0,False,2020-11-28 15:00:00,Gameweek 10,True,3,1,20,1
4,106,2128393,17,NaN,1,NaN,11.0,False,0,False,2020-12-05 15:00:00,Gameweek 11,False,4,1,17,1


In [25]:
def player_future(player_id):
    data = player_future_fixture[player_future_fixture.player_id == player_id]

    player = all_players[all_players.player_id == player_id]
    team = data.team_id.iloc[0]
    goals_scored = player.iloc[0].goals_scored
    player_name = player.iloc[0]['web_name']

    fig = px.line(
        data, 
        x = 'event_name', 
        y = 'difficulty', 
        title = player_name,
        hover_data=['opponent_team_id'])

    fig.show()

In [26]:
player_future(150)

In [27]:
    opponent_strength = player_future_fixture['difficulty']
    opponent_strength
    #opponent_strength = player_analysis.iloc[:,2]

0        4
1        3
2        3
3        3
4        4
        ..
18972    4
18973    2
18974    2
18975    2
18976    2
Name: difficulty, Length: 18977, dtype: int64

In [28]:
def get_SMA(player_id , count):
    data = player_future_fixture[player_future_fixture.player_id == player_id]
    opponent_strength = data['difficulty']
    # List to store moving average results
    SMA2 = list(range(0, len(opponent_strength) - count))

    # Compute moving average
    for ii in range(len(SMA2)):
        # Previous gw index
        index = range(ii, ii + count)
        # Get data for previous gw
        prev_gw2 = opponent_strength.iloc[index]

        # Sum previous gw
        summation2 = np.sum(prev_gw2)

        # Get average
        avg2 = summation2/count
        # Save results to list
        SMA2[ii] = avg2
         
    # Define column label
    label2 = f"Opponent strength SMA"

    
    # Get corresponding gw for moving_avg
    gameweeks2 = opponent_strength.index[:-count]
    gameweek = data.event_name[:-count]
    
    # Convert list into Pandas Series
    SMA2 = pd.Series(SMA2, name = label2, index = gameweek)
    
    
    return SMA2

In [29]:
def player_future_avg(player_id , no_games):
    data = get_SMA(player_id , no_games)

    player = all_players[all_players.player_id == player_id]

    player_name = player.iloc[0]['web_name']
    title = "Average difficulty next " + str(no_games) + " games for " + str(player_name)

    fig = px.line(
        data, 
        title = title)

    fig.show()

In [30]:
player_future_avg(200 , 6)

In [31]:
result = get_SMA(150,4)
result

event_name
Gameweek 7     2.75
Gameweek 8     2.50
Gameweek 9     2.50
Gameweek 10    3.00
Gameweek 11    3.50
Gameweek 12    3.75
Gameweek 13    3.75
Gameweek 14    3.50
Gameweek 15    3.25
Gameweek 16    3.25
Gameweek 17    3.00
Gameweek 18    2.75
Gameweek 19    2.75
Gameweek 20    3.00
Gameweek 21    2.75
Gameweek 22    3.50
Gameweek 23    3.25
Gameweek 24    3.25
Gameweek 25    3.25
Gameweek 26    2.50
Gameweek 27    2.50
Gameweek 28    2.25
Gameweek 29    2.75
Gameweek 30    3.25
Gameweek 31    3.50
Gameweek 32    3.50
Gameweek 33    3.25
Gameweek 34    3.00
Name: Opponent strength SMA, dtype: float64

## FANTASY PLAYER VALUE

In [32]:
all_players.columns

Index(['chance_of_playing_next_round', 'chance_of_playing_this_round',
       'player_code', 'cost_change_event', 'cost_change_event_fall',
       'cost_change_start', 'cost_change_start_fall', 'dreamteam_count',
       'player_type_id', 'ep_next', 'ep_this', 'event_points', 'form',
       'player_id', 'in_dreamteam', 'news', 'news_added', 'now_cost', 'photo',
       'PPG', 'selected_by_percent', 'status', 'team_id', 'team_code',
       'total_points', 'transfers_in_event', 'transfers_out_event',
       'value_form', 'value_season', 'web_name', 'total_minutes',
       'goals_scored', 'assists', 'clean_sheets', 'goals_conceded',
       'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards',
       'red_cards', 'saves', 'bonus', 'bps', 'influence', 'creativity',
       'threat', 'ict_index', 'influence_rank', 'influence_rank_type',
       'creativity_rank', 'creativity_rank_type', 'threat_rank',
       'threat_rank_type', 'ict_index_rank', 'ict_index_rank_type',
       'corne

In [33]:
top_players = all_players[['web_name' , 'now_cost', 'status' , 'player_type_id', 'total_points' , 'total_minutes' , 'PP90' , 'PPG' , 'PPMM' , 'VAPM']][all_players['total_minutes']>100][all_players['status'] != "i"].sort_values(by='VAPM' , ascending = False).head(60)
top_players.head(20)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
446,Kilman,41,a,2,23,270,7.666667,7.7,1.878049,1.390244
289,Chilwell,58,a,2,27,270,9.000000,9.0,1.551724,1.206897
16,Hourihane,60,a,3,17,137,11.167883,8.5,1.416667,1.083333
485,Bamford,59,a,4,49,490,9.000000,8.2,1.389831,1.050847
528,Martínez,48,a,1,34,450,6.800000,6.8,1.416667,1.000000
48,Jorginho,52,a,3,35,432,7.291667,7.0,1.346154,0.961538
540,Mendy,50,a,1,13,180,6.500000,6.5,1.300000,0.900000
411,Aurier,52,a,2,13,180,6.500000,6.5,1.250000,0.865385
284,Zouma,51,a,2,32,450,6.400000,6.4,1.254902,0.862745
360,Cancelo,54,a,2,13,180,6.500000,6.5,1.203704,0.833333


In [34]:
top_players[top_players['player_type_id'] == 1].head(3)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
528,Martínez,48,a,1,34,450,6.8,6.8,1.416667,1.0
540,Mendy,50,a,1,13,180,6.5,6.5,1.300000,0.9
568,Darlow,50,a,1,30,540,5.0,5.0,1.000000,0.6


In [35]:
top_players[top_players['player_type_id'] == 2].head(5)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
446,Kilman,41,a,2,23,270,7.666667,7.7,1.878049,1.390244
289,Chilwell,58,a,2,27,270,9.000000,9.0,1.551724,1.206897
411,Aurier,52,a,2,13,180,6.500000,6.5,1.250000,0.865385
284,Zouma,51,a,2,32,450,6.400000,6.4,1.254902,0.862745
360,Cancelo,54,a,2,13,180,6.500000,6.5,1.203704,0.833333


In [36]:
top_players[top_players['player_type_id'] == 3].head(5)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
16,Hourihane,60,a,3,17,137,11.167883,8.5,1.416667,1.083333
48,Jorginho,52,a,3,35,432,7.291667,7.0,1.346154,0.961538
62,Zaha,72,a,3,48,540,8.000000,8.0,1.111111,0.833333
192,Son,95,a,3,59,380,13.973684,9.8,1.031579,0.821053
18,Grealish,72,a,3,39,450,7.800000,7.8,1.083333,0.805556


In [37]:
top_players[top_players['player_type_id'] == 4].head(3)

,web_name,now_cost,status,player_type_id,total_points,total_minutes,PP90,PPG,PPMM,VAPM
485,Bamford,59,a,4,49,490,9.000000,8.2,1.389831,1.050847
510,Kane,109,a,4,61,447,12.281879,10.2,0.935780,0.752294
499,Wilson,65,a,4,41,534,6.910112,6.8,1.046154,0.738462


In [38]:
price_gk = top_players['now_cost'][top_players['player_type_id'] == 1].head(2).sum()
price_def = top_players['now_cost'][top_players['player_type_id'] == 2].head(5).sum()
price_mid = top_players['now_cost'][top_players['player_type_id'] == 3].head(5).sum()
price_fwd = top_players['now_cost'][top_players['player_type_id'] == 4].head(3).sum()


team_cost = price_gk + price_def + price_mid + price_fwd
print("Price for the best team is " + str(np.divide(team_cost,10)))

Price for the best team is 93.8


In [39]:
def player_value():

    fig = px.scatter(
        top_players.head(30), 
        x = 'now_cost', 
        y = 'VAPM',
        #size = 'minutes',
        color = 'total_points',
        text = 'web_name',
        hover_data=['PPG'])

    fig.show()

In [40]:
player_value()